In [20]:
import numpy as np
import pandas as pd
from collections import defaultdict
import os
import json
import shutil
from PIL import Image

import warnings 
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import Dataset
import cv2

from pycocotools.coco import COCO

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

# GPU 사용 가능 여부에 따라 device 정보 저장
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
gt_csv = pd.read_csv('/opt/ml/ViT-Adapter/segmentation/work_dirs/9_uper_adapter_beit_L_focal/vit_focal_20.csv', index_col=None) #
compare = pd.read_csv('/opt/ml/ViT-Adapter/segmentation/work_dirs/9_uper_adapter_beit_L_focal/vit_focal_20_pd.csv', index_col=None)

classes = [ "Background", "General trash", "Paper", "Paper pack", "Metal",
    "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing", ]

num_class = dict()
for i in range(len(classes)):
    num_class[i] = classes[i]

print(num_class)

{0: 'Background', 1: 'General trash', 2: 'Paper', 3: 'Paper pack', 4: 'Metal', 5: 'Glass', 6: 'Plastic', 7: 'Styrofoam', 8: 'Plastic bag', 9: 'Battery', 10: 'Clothing'}


In [9]:
# calculate mIoU
c = 0
pseudo_list = []
for i in range(len(gt_csv)):
    gts, pre = gt_csv['PredictionString'][i], compare['PredictionString'][i]
    gts_list, pre_list = list(map(int,gts.split())), list(map(int,pre.split()))
    IoU = []
    temp_union, temp_inter = defaultdict(int), defaultdict(int)
    
    for j in range(len(gts_list)):
        gt, pr = gts_list[j], pre_list[j]
        if gt == pr:
            temp_inter[gt] += 1
            temp_union[pr] += 1
        else:
            temp_union[gt] += 1
            temp_union[pr] += 1


    for key in temp_union.keys():
        IoU.append((temp_inter[key])/(temp_union[key]))
    
    mIoU = sum(IoU)/len(IoU)
    if mIoU > 0.8:
        pseudo_list.append(i)
        c += 1

print("total num : ", c/len(gt_csv))

total num :  0.2783882783882784


In [10]:
# maybe,, useless
def make_image_list(pseudo_list):
    image_list = []
    for i in range(len(gt_csv)):
        if i in pseudo_list:
            image_list.append(gt_csv['image_id'][i])
    return image_list

image_list = make_image_list(pseudo_list)

In [28]:
# make directory and copy images for pseudo labels
def _make_masks(pseudo_list, image_dir):
     for i in range(len(gt_csv)):
        if i in pseudo_list:
            # copy masks
            masks = np.array(list(map(int,gt_csv['PredictionString'][i].split())))
            masks = masks.reshape((512,512,1))
            # img = Image.fromarray(masks)
            cv2.imwrite(os.path.join(image_dir, f"{i:04}.png"), masks)
            break
    

def _rename_images(pseudo_list, image_dir):
    for i in range(len(gt_csv)):
        if i in pseudo_list:
            # copy images
            shutil.copyfile(
                os.path.join("/opt/ml/input/data", gt_csv['image_id'][i]),
                os.path.join(image_dir, f"{i:04}.jpg"),
            )


def make(pseudo_list, path):
    imagePath = "/opt/ml/input/data/" + path
    maskPath = "/opt/ml/input/data/" + (path+'_ano')
    os.makedirs(imagePath, exist_ok=True)
    os.makedirs(maskPath, exist_ok=True)
    _rename_images(pseudo_list, imagePath)
    # print(maskPath)
    _make_masks(pseudo_list, maskPath)
    print("psuedo folder and copy complete")

make(pseudo_list, 'pseudo')

psuedo folder and copy complete
